In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import yaml, sys, os
import pickle as pkl
import pulp, shutil, random

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)


import warnings
warnings.filterwarnings('ignore')

### Solve Integer Programming with Genetic Algorithm + Wisdom of Artificial Crowds

Integer Programming:

optimize $$c^T x$$

subject to $$ Ax \leq b$$

and $$x \geq 0, \; x \in Z$$

In [2]:
# read in the parameters for genetic algorithm from inputs.yml
with open('inputs.yml', 'rb') as f:
    params = yaml.safe_load(f.read())

problem_size = params['n']
preload_data_flag = params['data_load']

### Data preperation

In [3]:
class Initializer:
    def __init__(self, n, n_constraints=10):
        self.n = n
        self.n_constraints = n_constraints
    def create_vector_c(self):
        return np.random.randint(-10, 10, self.n)
    def create_matrix_A(self):
        return np.random.randint(-5, 10, (self.n_constraints, self.n))
    def create_vector_b(self):
        return np.random.randint(self.n*2, self.n*10, self.n_constraints)
    def save_initial_data(self, c, A, b, filename=None):
        with open(filename, 'wb') as f:
            pkl.dump({'c': c, 'A': A, 'b': b}, f)
    
def load_initial_data(filename):
    with open(filename, 'rb') as f:
        data = pkl.load(f)
    return data['c'], data['A'], data['b']

if preload_data_flag:
    c, A, b = load_initial_data(filename=f'data/linear_programming_data_{problem_size}.pkl')
else:
    num_constraints = max(int(problem_size/16), 5)
    initializer = Initializer(problem_size, num_constraints)
    c = initializer.create_vector_c()
    A = initializer.create_matrix_A()
    b = initializer.create_vector_b()
    initializer.save_initial_data(c, A, b, filename=f'data/linear_programming_data_{problem_size}.pkl')
    sys.exit("Data initialized and saved. Please set 'data_load' to True in inputs.yml to load the data next time.")

print(A.shape)
print(b.shape)
print(c.shape)

(5, 16)
(5,)
(16,)


In [4]:
def solve_integer_programming(c, A, b, method='cbc'):
    """"
    Maximize cᵀx
    subject to Ax ≤ b
    and x ≥ 0, x ∈ Z
    """
    c = np.array(c)
    A = np.array(A)
    b = np.array(b)
    n_vars = len(c)
    n_constraints = len(b)
    # 1. Create a linear programming model (using PuLP)
    model = pulp.LpProblem("MatrixForm_LP", pulp.LpMaximize)
    # 2. Create optimization, integer variables 0 <= x0, x1, x2, ... <= 50
    x = pulp.LpVariable.dicts("x", range(n_vars), lowBound=0, upBound=50, cat="Integer")
    # 3. Objective: maximize cᵀx
    model += pulp.lpSum(c[j] * x[j] for j in range(n_vars))
    # 4. Constraints: A_i * x ≤ b_i
    for i in range(n_constraints):
        model += pulp.lpSum(A[i, j] * x[j] for j in range(n_vars)) <= b[i]
    # 5. Solve
    if method=='cbc':
        # Dual/Primal Simplex
        model.solve(pulp.COIN_CMD(path="/opt/homebrew/bin/cbc", msg=False))
    elif method=='glpk':
        # simplex + Interior Point
        model.solve(pulp.GLPK_CMD(msg=False))
    y = pulp.value(model.objective)
    return x, y

# Solve the integer programming:
# Maximize cᵀx
# subject to Ax ≤ b
# and x ≥ 0, x ∈ Z
x, obj_value = solve_integer_programming(c, A, b, method='cbc')
print("Objective value =", obj_value)
# prepare data
results_data = {
    "Variable": [f"x{j}" for j in range(problem_size)],
    "Value": [x[j].value() for j in range(problem_size)]
}
df = pd.DataFrame(results_data)
# df = df.sort_values("Variable").reset_index(drop=True)
df.to_pickle(f"data/optimal_solution_{problem_size}.pkl")
df[df["Value"] > 0].head(problem_size)

Objective value = 101.0


,Variable,Value
1,x1,13.0
5,x5,4.0
8,x8,19.0


### Genetic Algorithm

In [5]:
class GeneticAlgorithmWOC():
    def __init__(self, c, A, b, lb, ub):
        self.c = c
        self.A = A
        self.b = b
        with open('inputs.yml', 'rb') as f:
            params = yaml.safe_load(f.read())
        self.population_size = params['population_size']
        self.n_gen = params['num_generations']
        self.tournament_size = params['tournament_size']
        self.inv_p = params['inv_p']
        self.swap_p= params['inv_p']
        self.topsoln_frac = params['topsoln_frac']
        self.keep_children = params['keep_children']
        self.stall_patience = params['stall_patience']
        self.n_vars = len(c)
        self.n_constraints = len(b)
        self.lower_bound = lb
        self.upper_bound = ub

    def constraint_function(self, solution):
        """
        Check if a solution satisfies the constraints Ax <= b.
        Returns True if feasible, False otherwise.
        """
        return all(np.dot(self.A[i], solution) <= self.b[i] for i in range(self.n_constraints))

    def fitness_function(self, solution):
        """
        Calculate the fitness of a solution.
        Fitness is defined as the objective value if constraints are satisfied,
        otherwise a large negative penalty is applied.
        """
        obj_value = np.dot(self.c, solution)
        constraints_satisfied = self.constraint_function(solution)
        if constraints_satisfied:
            return obj_value
        else:
            penalty = -1e6  # Large negative penalty for infeasible solutions
            return penalty

    @staticmethod
    def is_feasible(x, A, b):
        """
        Quick check to see if Ax <= b (constraints)
        """
        x_arr = np.array(x)
        return np.all(A @ x_arr <= b + 1e-9)

    def generate_feasible_individual(self):
        """
        Greedy constructive heuristic to generate feasible solution x in [lb, ub]^n s.t. A x <= b is satisfied.
        """
        n = self.n_vars
        lb = int(self.lower_bound)
        ub = int(self.upper_bound)
        x = np.zeros(n, dtype=int)
        slack = self.b.astype(float) - self.A @ x  # start with full slack = b
        order = np.random.permutation(n)

        for j in order:
            col = self.A[:, j].astype(float)
            pos_mask = col > 0

            if np.any(pos_mask):
                vmax_list = np.floor(slack[pos_mask] / col[pos_mask]).astype(int)
                vmax = int(vmax_list.min()) if vmax_list.size > 0 else ub
            else:
                # If all coefficients <= 0, increasing this variable doesn't hurt feasibility
                vmax = ub

            vmax = min(max(vmax, lb), ub)
            vmin = lb

            if vmax < vmin:
                # No room to increase without violating constraints; pick the safest value
                val = vmin if vmin == 0 else 0
            else:
                val = np.random.randint(vmin, vmax + 1)

            x[j] = int(val)
            slack = slack - col * val

        # # Repair if any slight violations remain
        # if not np.all(slack >= -1e-9):
        #     for i in range(self.n_constraints):
        #         if slack[i] < 0:
        #             # reduce variables that contribute positively to the violated constraint
        #             while slack[i] < 0:
        #                 candidates = [j for j in range(n) if self.A[i, j] > 0 and x[j] > lb]
        #                 if not candidates:
        #                     break
        #                 j = int(np.random.choice(candidates))
        #                 reduce_by = int(np.ceil((-slack[i]) / self.A[i, j]))
        #                 new_val = max(lb, x[j] - reduce_by)
        #                 delta = x[j] - new_val
        #                 x[j] = new_val
        #                 slack = self.b.astype(float) - self.A @ x

        return x.tolist()

    def create_init_popn(self):
        """
        Create an initial population of feasible solutions (A x <= b) with x integer in [lb, ub].
        """
        initial_pop = []
        lb = int(self.lower_bound)
        ub = int(self.upper_bound)
        tries_per_individual = 10
        for _ in range(self.population_size):
            x = None
            # Try greedy construction multiple times with different orders
            for __ in range(tries_per_individual):
                candidate = self.generate_feasible_individual()
                if self.is_feasible(candidate, self.A, self.b):
                    x = candidate
                    break
            initial_pop.append(x)
        return initial_pop
    
    def choose_best_individuals(self, population, fitnesses):
        """
        Select the top fraction of individuals based on fitness.
        """
        num_top = max(1, int(self.topsoln_frac * len(population)))
        sorted_indices = np.argsort(fitnesses)[::-1]
        best_indices = sorted_indices[:num_top]
        best_individuals = [population[i] for i in best_indices]
        return best_individuals
    
    def choose_random_best_solution(self, population, fitnesses, k=5):
        n = len(population)
        idx = random.sample(range(n), k)
        best = max(idx, key=lambda i: fitnesses[i])
        return population[best]
    
    def ox(self,p1,p2):
        n = self.n_vars
        assert len(p1) == n and len(p2) == n, "Parents must have length n_vars"
        lb = int(self.lower_bound)
        ub = int(self.upper_bound)

        def _max_feasible_for_var(slack, col, lb, ub):
            pos = col > 0
            if np.any(pos):
                vmax_list = np.floor(slack[pos] / col[pos]).astype(int)
                vmax = int(vmax_list.min()) if vmax_list.size > 0 else ub
            else:
                vmax = ub
            return min(ub, max(lb, vmax))


        i, j = sorted(np.random.choice(np.arange(n), size=2, replace=False))

        def build_child(keep, fill):
            x = np.zeros(n, dtype=int)
            slack = self.b.astype(float) - self.A @ x
            # 1) copy the middle slice from 'keep' but cap by feasibility
            for k in range(i, j + 1):
                col = self.A[:, k].astype(float)
                desired = int(keep[k])
                vmax = _max_feasible_for_var(slack, col, lb, ub)
                val = max(lb, min(desired, vmax))
                x[k] = val
                slack = slack - col * val
            # 2) fill remaining positions in cyclic order from 'fill'
            for k in list(range(j + 1, n)) + list(range(0, i)):
                col = self.A[:, k].astype(float)
                desired = int(fill[k])
                vmax = _max_feasible_for_var(slack, col, lb, ub)
                val = max(lb, min(desired, vmax))
                x[k] = val
                slack = slack - col * val
            # # 3) final repair if any violations remain (should be rare)
            # if not np.all(slack >= -1e-9):
            #     for ci in range(self.n_constraints):
            #         while slack[ci] < -1e-9:
            #             candidates = [jj for jj in range(n) if self.A[ci, jj] > 0 and x[jj] > lb]
            #             if not candidates:
            #                 break
            #             jj = int(np.random.choice(candidates))
            #             reduce_by = int(np.ceil((-slack[ci]) / self.A[ci, jj]))
            #             new_val = max(lb, x[jj] - reduce_by)
            #             x[jj] = new_val
            #             slack = self.b.astype(float) - self.A @ x
            # if not self.is_feasible(x, self.A, self.b):
            #     return self.generate_feasible_individual()
            return x.tolist()

        child1 = build_child(p1, p2)
        child2 = build_child(p2, p1)
        return child1, child2
    
    def genetic_alg_main(self):
        """
        Main loop for the genetic algorithm without crossover.
        """
        # 1. Create initial population
        population = self.create_init_popn()
        best_fitness_over_time = []
        fitness = [self.fitness_function(ind) for ind in population]
        best_fitness = min(fitness)

        stall = 0
        hist_best, hist_mean = [], []
        for gen in range(self.n_gen):
            curr_best_fitness = min(fitness)
            curr_mean = np.mean(fitness)
            hist_best.append(curr_best_fitness)
            hist_mean.append(curr_mean)
            if curr_best_fitness < best_fitness:
                best_fitness = curr_best_fitness
                best_solution= population[fitness.index[best_fitness]]
                stall = 0
            else:
                stall += 1
                if stall >= self.stall_patience: break

            # find next population
            next_population = self.choose_best_individuals(population, fitness)
            while len(next_population) < self.population_size:
                # select 2 parents using choose_random_best_solution
                p1 = self.choose_random_best_solution(population, fitness)
                p2 = self.choose_random_best_solution(population, fitness)
                while p1 == p2:
                    p2 = self.choose_random_best_solution(population, fitness)
                ox1, ox2 = self.ox(p1,p2)

                sys.exit()

        


        # for gen in range(self.n_gen):
        #     # 2. Evaluate fitness
        #     fitnesses = [self.fitness_function(ind) for ind in population]
        #     best_fitness = max(fitnesses)
        #     best_fitness_over_time.append(best_fitness)
        #     print(f"Generation {gen}: Best Fitness = {best_fitness}")
        #     # 3. Select best individuals
        #     best_individuals = self.choose_best_individuals(population, fitnesses)
        #     # 4. Create new population
        #     new_population = best_individuals.copy()
        #     while len(new_population) < self.population_size:
        #         parent = np.random.choice(best_individuals)
        #         child = parent.copy()
        #         # Mutation: random change in one variable
        #         mutate_index = np.random.randint(0, self.n_vars)
        #         mutate_value = np.random.randint(int(self.lower_bound), int(self.upper_bound) + 1)
        #         child[mutate_index] = mutate_value
        #         if self.is_feasible(child, self.A, self.b):
        #             new_population.append(child)
        #     population = new_population
        # return best_fitness_over_time


In [7]:
ga = GeneticAlgorithmWOC(c, A, b, lb=0, ub=50)
init_pop = initial_population = ga.create_init_popn()
ga.genetic_alg_main()


SystemExit: 